In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel('/content/drive/MyDrive/Cs549/CS549_project.xlsx')

df = df[['Stock_Report', 'News']]

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel('/content/drive/MyDrive/Cs549/CS549_project.xlsx')
df = df[['Stock_Report', 'News']].dropna()

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df)
})

from transformers import MT5ForConditionalGeneration, MT5Tokenizer

tokenizer = MT5Tokenizer.from_pretrained('ozcangundes/mt5-small-turkish-summarization')
model = MT5ForConditionalGeneration.from_pretrained('ozcangundes/mt5-small-turkish-summarization')

def tokenize_function(examples):
    inputs = ["haberleştir: " + text for text in examples["Stock_Report"]]
    targets = examples["News"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['Stock_Report', 'News'])

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./kap_news_model',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=70,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    generation_max_length=256,
    logging_dir='./logs',
    logging_steps=10,
    push_to_hub=False
)

from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.286300,2.196536
2,2.103800,1.759583
3,1.959800,1.675701
4,1.755400,1.631280
5,1.668900,1.596638
6,1.679900,1.576433
7,1.692300,1.557438
8,1.683700,1.540171
9,1.467200,1.522428
10,1.562200,1.514502


Checkpoint destination directory ./kap_news_model/checkpoint-4500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=6300, training_loss=1.243396153222947, metrics={'train_runtime': 1136.0506, 'train_samples_per_second': 22.182, 'train_steps_per_second': 5.546, 'total_flos': 1.3324492210176e+16, 'train_loss': 1.243396153222947, 'epoch': 70.0})

In [ ]:
def generate_news(stock_report_text):
    input_text = "haberleştir: " + stock_report_text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=256,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=2.0,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

example_text = "Şirketimizin 21.03.2025 tarihli özel durum açıklamasında, paylarının tamamına sahip olduğumuz bağlı ortaklığımız Marbaş Menkul Değerler AŞ'nin (Marbaş Menkul), 150.000.000 TL olan çıkarılmış sermayesinin 360.000.000 TL'ye çıkarılmasına, artırılan 210.000.000 TL sermayenin iç kaynaklardan karşılanmasına ilişkin olarak Marbaş Menkul tarafından Sermaye Piyasası Kurulu'na yapılan başvurunun olumlu karşılandığı duyurulmuştur."
print("📰 Oluşturulan Haber:\n", generate_news(example_text))


📰 Oluşturulan Haber:
 Marbaş Menkul Değerler A.Ş., bağlı ortaklığı Marbaş Menkul Değerler A.Ş.’nin sermayesini 220 milyon TL’ye çıkarma kararı aldı. 150 milyon TL olan çıkarılmış sermaye, iç kaynaklardan karşılanacak ve iç kaynaklardan karşılanacak. Sermaye Piyasası Kurulu’na yapılan başvuruyu olumlu karşılayarak Marbaş tarafından yapılan başvurunun olumlu karşılandığı bildirildi. Bu gelişme, Marbaş tarafından yapılan başvuruda olumlu karşılandı. Artışın tamamına sahip olduğu bağlı ortaklık Marbaş Menkul Değerler A.Ş.’nin sermaye artırımı sürecine ilişkin önemli bir adım olarak değerlendiriliyor. Açıkçası bu hamle, hem finansal yapıyı desteklemek hem de yatırımcıya güven vermek açısından oldukça pozitif bir gelişme olmuş.


In [ ]:
example_text = "Sermayesine %49 oranında ortaklığımızın bulunduğu iştiraklerimiz Sour Turizm A.Ş. ve Çağrankaya Turizm A.Ş.'nin sermayelerinin 250.000.000 TL'den 400.000.000 TL'ye çıkarılmasına karar verilmiştir."
print("📰 Oluşturulan Haber:\n", generate_news(example_text))



📰 Oluşturulan Haber:
 Şirket, %49 oranında ortaklığı olan Sour Turizm ve Çağrankaya Turizm A.Ş.’nin sermayesini 250 milyon TL’den 40 milyon TL’ye çıkarma kararı aldı. 25 milyon TL’lik pay sahiplerinin bulunduğu iştiraklerden biri olan Sour Turizm ve Çağrankaya Turizm’in sermayesi ise 400 milyon TL’ye çıkarılacak. Bu hamle, şirketin büyüme stratejisinin bir parçası olarak değerlendiriliyor. Sermayenin %49 oranında ortaklığı bulunduğu iştiraklerinden biri olan Sour Turizm ve Çağrankaya Turizm’in borçlanma şirketleri tarafından sağlanacak. Açıkçası bu gelişme, uzun vadeli yatırımcılar açısından oldukça değerli bir adım olmuş.


In [ ]:
!pip install accelerate==0.28.0


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!pip install transformers==4.38.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


In [ ]:
!pip uninstall -y transformers tokenizers huggingface_hub


Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
Found existing installation: huggingface-hub 0.31.4
Uninstalling huggingface-hub-0.31.4:
  Successfully uninstalled huggingface-hub-0.31.4


In [ ]:
import transformers
print(transformers.__version__)


4.38.2
